# Sujet
L'entreprise de produits pharmaceutiques HumanForYou basée en Inde emploie environ 4000 personnes. Cependant, chaque année elle subit un turn-over d'environ 15% de ses employés nécessitant de retrouver des profils similaires sur le marché de l'emploi.
La direction trouve que ce niveau de turn-over n'est pas bon pour l'entreprise car :
•	Les projets sur lesquels étaient les employés quittant la société prennent du retard ce qui nuit à la réputation de l'entreprise auprès de ses clients et partenaires.
•	Un service de ressources humaines de taille conséquente doit être conservé car il faut avoir les moyens de trouver les nouvelles recrues.
•	Du temps est perdu à l'arrivée des nouveaux employés car ils doivent très souvent être formés et ont besoin de temps pour devenir pleinement opérationnels dans leur nouvel environnement.
Le direction fait donc appel à vous, spécialistes de l'analyse de données, pour déterminer les facteurs ayant le plus d'influence sur ce taux de turn-over et lui proposer des modèles afin d'avoir des pistes d'amélioration pour donner à leurs employés l'envie de rester.


# Etape 1 : Préparation des données

In [1]:
# Import libraries
import pandas as pd, numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Import csv files
in_time_csv = pd.read_csv('./Base Data/in_time.csv')
out_time_csv = pd.read_csv('./Base Data/out_time.csv')
general_data_csv = pd.read_csv('./Base Data/general_data.csv')
manager_survey_data_csv = pd.read_csv('./Base Data/manager_survey_data.csv')
employee_survey_data_csv = pd.read_csv('./Base Data/employee_survey_data.csv')

Préparation des données de pointage :
- On supprime les valeurs manquantes (NaN) qui correspondent à un non-pointage
- On supprime les colonnes composées uniquement de NaN
- On transforme les pointages en quantité horaires travaillées

In [3]:
# Process in and out times
in_time_csv.shape
#in_time_csv.head

(4410, 262)

In [4]:
# Append the two csv files, convert all fields except the header and the first two columns to pandas datetime
total_time_csv = in_time_csv.append(out_time_csv, ignore_index=True)
total_time_csv = total_time_csv.replace(np.nan, 0)
total_time_csv.iloc[:, 1:] = total_time_csv.iloc[:, 1:].apply(pd.to_datetime, errors='coerce')
total_time_csv = total_time_csv.diff(periods=4410, axis=0)
total_time_csv = total_time_csv.iloc[4410:]
total_time_csv.reset_index(inplace=True)
total_time_csv.drop(columns=['Unnamed: 0', '2015-01-01', '2015-01-14','2015-01-26','2015-03-05',
             '2015-05-01','2015-07-17','2015-09-17','2015-10-02',
              '2015-11-09','2015-11-10','2015-11-11','2015-12-25', 'index'
             ],axis=1,inplace=True)
total_time_csv['Mean Time']=total_time_csv.mean(axis=1)
total_time_csv['hrs']=total_time_csv['Mean Time']/np.timedelta64(1, 'h')
total_time_csv.reset_index(inplace=True)
total_time_csv.drop(total_time_csv.columns.difference(['index','hrs']), 1, inplace=True)
total_time_csv.rename(columns={'index': 'EmployeeID'},inplace=True)

total_time_csv

,EmployeeID,hrs
0,0,6.870229
1,1,7.315971
2,2,6.816081
3,3,6.789215
4,4,7.877561
...,...,...
4405,4405,8.316921
4406,4406,5.897197
4407,4407,7.149526
4408,4408,9.187612


Création du fichier de données complètes :

In [5]:
# Merge the multiple csv files
complete_csv = pd.merge(general_data_csv, total_time_csv, on='EmployeeID', how='inner')
complete_csv = pd.merge(complete_csv, manager_survey_data_csv, on='EmployeeID', how='inner')
complete_csv = pd.merge(complete_csv, employee_survey_data_csv, on='EmployeeID', how='inner')
complete_csv.describe()

,Age,DistanceFromHome,Education,EmployeeCount,EmployeeID,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,hrs,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
count,4409.000000,4409.000000,4409.000000,4409.0,4409.000000,4409.000000,4409.000000,4390.000000,4409.000000,4409.0,...,4409.000000,4409.000000,4409.000000,4409.000000,4409.000000,4409.000000,4409.000000,4384.000000,4389.000000,4372.000000
mean,36.923112,9.188251,2.912905,1.0,2205.000000,2.063960,65031.660240,2.695444,15.210252,8.0,...,2.798594,7.004990,2.187571,4.122023,7.314463,2.729644,3.153776,2.723996,2.728184,2.761436
std,9.134220,8.100992,1.024048,0.0,1272.912998,1.106814,47073.969139,2.498840,3.659203,0.0,...,1.288222,6.122203,3.222042,3.566975,1.330903,0.711223,0.360775,1.092571,1.101371,0.706245
min,18.000000,1.000000,1.000000,1.0,1.000000,1.000000,10090.000000,0.000000,11.000000,8.0,...,0.000000,0.000000,0.000000,0.000000,5.416880,1.000000,3.000000,1.000000,1.000000,1.000000
25%,30.000000,2.000000,2.000000,1.0,1103.000000,1.000000,29110.000000,1.000000,12.000000,8.0,...,2.000000,3.000000,0.000000,2.000000,6.278995,2.000000,3.000000,2.000000,2.000000,2.000000
50%,36.000000,7.000000,3.000000,1.0,2205.000000,2.000000,49080.000000,2.000000,14.000000,8.0,...,3.000000,5.000000,1.000000,3.000000,7.011237,3.000000,3.000000,3.000000,3.000000,3.000000
75%,43.000000,14.000000,4.000000,1.0,3307.000000,3.000000,83800.000000,4.000000,18.000000,8.0,...,3.000000,9.000000,3.000000,7.000000,7.900932,3.000000,3.000000,4.000000,4.000000,3.000000
max,60.000000,29.000000,5.000000,1.0,4409.000000,5.000000,199990.000000,9.000000,25.000000,8.0,...,6.000000,40.000000,15.000000,17.000000,10.937261,4.000000,4.000000,4.000000,4.000000,4.000000


Suppression des colonnes inutiles :

In [6]:
# Delete all print columns with always the same value
for col in complete_csv.columns:
    if len(complete_csv[col].unique()) == 1:
        complete_csv.drop(columns=[col],axis=1,inplace=True)
        print("Deleted : " + col + " (Cause : All values are the same)")

# The employee ID is not useful for the analysis
complete_csv.drop(columns=['EmployeeID'],axis=1,inplace=True)
print("Deleted : EmployeeID (Cause : Iteration useful for the analysis)")


Deleted : EmployeeCount (Cause : All values are the same)
Deleted : Over18 (Cause : All values are the same)
Deleted : StandardHours (Cause : All values are the same)
Deleted : EmployeeID (Cause : Iteration useful for the analysis)


Remplissage des données manquantes :

In [7]:
# Print the list of columns with NaN values and their type
for col in complete_csv.columns:
    if complete_csv[col].isna().values.any():
        print("Column with NaN : " + col + " (Type : " + str(complete_csv[col].dtype) + ")")

Column with NaN : NumCompaniesWorked (Type : float64)
Column with NaN : TotalWorkingYears (Type : float64)
Column with NaN : EnvironmentSatisfaction (Type : float64)
Column with NaN : JobSatisfaction (Type : float64)
Column with NaN : WorkLifeBalance (Type : float64)


Toutes ces colonnes peuvent avoir leurs valeurs manquantes remplacées par la médiane de la colonne car elles sont toutes numériques.

In [8]:
# Fill all numerical columns with the median of the column and print the list of the filled columns if they have nan values.
for col in complete_csv.columns:
    if complete_csv[col].isna().values.any():
        complete_csv[col].fillna(complete_csv[col].median(), inplace=True)
        print("Column with NaN filled with the median value : " + col)
#Verify if there is still NaN values
if complete_csv.isnull().values.any():
    print("There is still NaN values in the dataset.")
else:
    print("Dataset has been completely filled.")

Column with NaN filled with the median value : NumCompaniesWorked
Column with NaN filled with the median value : TotalWorkingYears
Column with NaN filled with the median value : EnvironmentSatisfaction
Column with NaN filled with the median value : JobSatisfaction
Column with NaN filled with the median value : WorkLifeBalance
Dataset has been completely filled.


Correction des types de données incorrects :
Ces données ne sont pas représentatives "linéairement", il faut donc les passer en type "object" pour éviter des erreurs lors de l'analyse.

In [9]:
complete_csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4409 entries, 0 to 4408
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Age                      4409 non-null   int64  
 1   Attrition                4409 non-null   object 
 2   BusinessTravel           4409 non-null   object 
 3   Department               4409 non-null   object 
 4   DistanceFromHome         4409 non-null   int64  
 5   Education                4409 non-null   int64  
 6   EducationField           4409 non-null   object 
 7   Gender                   4409 non-null   object 
 8   JobLevel                 4409 non-null   int64  
 9   JobRole                  4409 non-null   object 
 10  MaritalStatus            4409 non-null   object 
 11  MonthlyIncome            4409 non-null   int64  
 12  NumCompaniesWorked       4409 non-null   float64
 13  PercentSalaryHike        4409 non-null   int64  
 14  StockOptionLevel        

In [10]:
complete_csv['Education'] = complete_csv['Education'].astype('object')
complete_csv['EnvironmentSatisfaction'] = complete_csv['EnvironmentSatisfaction'].astype('object')
complete_csv['JobInvolvement'] = complete_csv['JobInvolvement'].astype('object')
complete_csv['JobSatisfaction'] = complete_csv['JobSatisfaction'].astype('object')
complete_csv['PerformanceRating'] = complete_csv['PerformanceRating'].astype('object')
complete_csv['JobLevel'] = complete_csv['JobLevel'].astype('object')
complete_csv['WorkLifeBalance'] = complete_csv['WorkLifeBalance'].astype('object')
complete_csv

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,hrs,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,No,Travel_Rarely,Sales,6,2,Life Sciences,Female,1,Healthcare Representative,...,6,1,0,0,7.315971,3,3,3.0,4.0,2.0
1,31,Yes,Travel_Frequently,Research & Development,10,1,Life Sciences,Female,1,Research Scientist,...,3,5,1,4,6.816081,2,4,3.0,2.0,4.0
2,32,No,Travel_Frequently,Research & Development,17,4,Other,Male,4,Sales Executive,...,2,5,0,3,6.789215,3,3,2.0,2.0,1.0
3,38,No,Non-Travel,Research & Development,2,5,Life Sciences,Male,3,Human Resources,...,5,8,7,5,7.877561,2,3,4.0,4.0,3.0
4,32,No,Travel_Rarely,Research & Development,10,1,Medical,Male,1,Sales Executive,...,2,6,0,4,10.275802,3,3,4.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,29,No,Travel_Rarely,Sales,4,3,Other,Female,2,Human Resources,...,2,6,1,5,8.316921,2,3,3.0,4.0,3.0
4405,42,No,Travel_Rarely,Research & Development,5,4,Medical,Female,1,Research Scientist,...,5,3,0,2,5.897197,3,3,4.0,1.0,3.0
4406,29,No,Travel_Rarely,Research & Development,2,4,Medical,Male,1,Laboratory Technician,...,2,3,0,2,7.149526,2,3,4.0,4.0,3.0
4407,25,No,Travel_Rarely,Research & Development,25,2,Life Sciences,Male,2,Sales Executive,...,4,4,1,2,9.187612,3,4,1.0,3.0,3.0


Remplacement des valeurs numériques par des valeurs textuelles :
Le jeu de données est assez propre, il ne reste plus qu'à remplacer certaines valeurs numériques par des valeurs textuelles pour faciliter l'analyse.

In [11]:
# Replace numerical values by text values
# general_data
complete_csv['Education'].replace({1: 'BAC', 2: 'BAC+2', 3: 'BAC+3', 4: 'BAC+5', 5: 'Doctorat'}, inplace=True)
# manager_survey
complete_csv['JobInvolvement'].replace({1: 'Faible', 2: 'Moyenne', 3: 'Importante', 4: 'Très importante'}, inplace=True)
complete_csv['PerformanceRating'].replace({1: 'Faible', 2: 'Bon', 3: 'Excellent', 4: 'Au delà des attentes'}, inplace=True)
# employee_survey
complete_csv['EnvironmentSatisfaction'].replace({1: 'Faible', 2: 'Moyen', 3: 'Élevé', 4: 'Très élevé'}, inplace=True)
complete_csv['JobSatisfaction'].replace({1: 'Faible', 2: 'Moyen', 3: 'Élevé', 4: 'Très élevé'}, inplace=True)
complete_csv['WorkLifeBalance'].replace({1: 'Mauvais', 2: 'Satisfaisant', 3: 'Très satisfaisant', 4: 'Excellent'}, inplace=True)

In [12]:
complete_csv

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,...,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,hrs,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance
0,51,No,Travel_Rarely,Sales,6,BAC+2,Life Sciences,Female,1,Healthcare Representative,...,6,1,0,0,7.315971,Importante,Excellent,Élevé,Très élevé,Satisfaisant
1,31,Yes,Travel_Frequently,Research & Development,10,BAC,Life Sciences,Female,1,Research Scientist,...,3,5,1,4,6.816081,Moyenne,Au delà des attentes,Élevé,Moyen,Excellent
2,32,No,Travel_Frequently,Research & Development,17,BAC+5,Other,Male,4,Sales Executive,...,2,5,0,3,6.789215,Importante,Excellent,Moyen,Moyen,Mauvais
3,38,No,Non-Travel,Research & Development,2,Doctorat,Life Sciences,Male,3,Human Resources,...,5,8,7,5,7.877561,Moyenne,Excellent,Très élevé,Très élevé,Très satisfaisant
4,32,No,Travel_Rarely,Research & Development,10,BAC,Medical,Male,1,Sales Executive,...,2,6,0,4,10.275802,Importante,Excellent,Très élevé,Faible,Très satisfaisant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4404,29,No,Travel_Rarely,Sales,4,BAC+3,Other,Female,2,Human Resources,...,2,6,1,5,8.316921,Moyenne,Excellent,Élevé,Très élevé,Très satisfaisant
4405,42,No,Travel_Rarely,Research & Development,5,BAC+5,Medical,Female,1,Research Scientist,...,5,3,0,2,5.897197,Importante,Excellent,Très élevé,Faible,Très satisfaisant
4406,29,No,Travel_Rarely,Research & Development,2,BAC+5,Medical,Male,1,Laboratory Technician,...,2,3,0,2,7.149526,Moyenne,Excellent,Très élevé,Très élevé,Très satisfaisant
4407,25,No,Travel_Rarely,Research & Development,25,BAC+2,Life Sciences,Male,2,Sales Executive,...,4,4,1,2,9.187612,Importante,Au delà des attentes,Faible,Élevé,Très satisfaisant


# Etape 2 : Analyse des données

Pour l'analyse des données, nous allons faire une copie du je ude données pour y ajouter une colonne temporaire.

In [83]:
# Create a copy of the dataset
analysis_dataset = complete_csv.copy()
# Adding a "Count" column with always "1" value
analysis_dataset['Count'] = 1
# Adding a "Count_Percentage" column which sum will be 100%
analysis_dataset['Count_Percentage'] = (1/analysis_dataset.shape[0])*100

Analyse des résultats de l'enquête manager par rapport à l'attrition :

In [113]:
from ipyvizzu import Data, Config
from ipyvizzustory import Story, Slide, Step
data = Data()
data.add_data_frame(analysis_dataset)
story = Story(data=data)
story.set_size("100%", "800px")

slide1 = Slide(
    Step(
        Config(
            {
                "channels": {
                    "x": ["Count", "Attrition"],
                    "y": "JobInvolvement",
                    "noop": "Attrition",
                },
                "title": "Implication dans le travail",
            }
        )
    )
)

story.add_slide(slide1)

slide2 = Slide()
slide2.add_step(
    Step(
        Config(
            {
                "channels": {
                    "x": {"range": {"max": "100%"}},
                    "color": "Attrition",
                    "noop": None,
                    "label": "Count",
                },
                "title": "Implication dans le travail",
                "split": True,
            }
        )
    )
)

slide2.add_step(
    Step(
        Config(
            {
                "channels": {"x": {"range": {"max": "auto"}}},
                "title": "Implication dans le travail",
                "split": False,

            }
        )
    )
)

slide2.add_step(
    Step(
        Config({"title": "Implication dans le travail", "align": "stretch"})
    )
)

story.add_slide(slide2)


slide3 = Slide(
    Step(
        Data.filter("record.Attrition === 'Yes'"),
        Config(
            {
                "channels": {
                    "x": ["Count","JobInvolvement"],
                    "y": "Attrition",
                    "noop": None,
                    "color": "JobInvolvement",
                },
                "title": "Implication dans le travail",
                "split": False,
            }
        )
    )
)

story.add_slide(slide3)

slide4 = Slide()

slide4.add_step(
    Step(
        Data.filter(None),
        Config(
            {
                "channels": {
                    "x": "PerformanceRating",
                    "y": ["Count", "Attrition"],
                    "noop": "Attrition",
                    "color": None,
                },
                "title": "Niveau de performance",
                "split": False,
            }
        )
    )
)

slide4.add_step(
    Step(
        Config(
            {
                "channels": {
                    "y": {"range": {"max": "100%"}},
                    "color": "Attrition",
                    "noop": None,
                    "label": "Count",
                },
                "title": "Niveau de performance",
                "split": True,
            }
        )
    )
)

slide4.add_step(
    Step(
        Config(
            {
                "channels": {"y": {"range": {"max": "auto"}}},
                "title": "Niveau de performance",
                "split": False,
            }
        )
    )
)

slide4.add_step(
    Step(
        Config({"title": "Niveau de performance", "align": "stretch"})
    )
)

slide5 = Slide(
    Step(
        Data.filter("record.Attrition === 'Yes'"),
        Config(
            {
                "channels": {
                    "x": ["Count","PerformanceRating"],
                    "y": "Attrition",
                    "noop": None,
                    "color": "PerformanceRating",
                },
                "title": "Niveau de performance",
                "split": False,
            }
        )
    )
)

story.add_slide(slide4)
story.add_slide(slide5)

story.set_feature("tooltip", True)

story.play()